<a href="https://colab.research.google.com/github/blackwithwhitegreen/Langchain/blob/main/Retriever_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain chromadb faiss-cpu openai tiktoken langchain_openai langchain-community wikipedia

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from langchain_community.retrievers import WikipediaRetriever

In [3]:
# Initialize the retriever (optional: set language and top_k)
retriever = WikipediaRetriever(top_k_results=2, lang="en")

In [4]:
# Defien Query
query  = "The geopolitics history of India and pakistan from perspective of a Chinese"

# Get relevent Wikipidia documents
docs = retriever.invoke(query)

In [5]:
# Print retrived content
for i, doc in enumerate(docs):
  print(f"\n---Results {i+1} ----")
  print(f"Content:\n {doc.page_content}....")


---Results 1 ----
Content:
 The history of independent India or history of Republic of India began when the country became an independent sovereign state within the British Commonwealth on 15 August 1947. Direct administration by the British, which began in 1858, affected a political and economic unification of the subcontinent. When British rule came to an end in 1947, the subcontinent was partitioned along religious lines into two separate countries—India, with a majority of Hindus, and Pakistan, with a majority of Muslims. Concurrently the Muslim-majority northwest and east of British India was separated into the Dominion of Pakistan, by the Partition of India. The partition led to a population transfer of more than 10 million people between India and Pakistan and the death of about one million people. Indian National Congress leader Jawaharlal Nehru became the first Prime Minister of India, but the leader most associated with the independence struggle, Mahatma Gandhi, accepted no 

# Vector Store Retriever

In [18]:
from langchain_community.vectorstores import Chroma
from langchain_openai import  OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_community.embeddings import SentenceTransformerEmbeddings

In [19]:
# Step 1: Your source documnets
documents = [
    Document(page_content="Langchain helps developers build LLM application easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embedding convert text into high-dimensional vectores."),
    Document(page_content="OpenAI provies powerful embedding models."),


]

In [20]:
# Initialize embedding model
embedding_model = SentenceTransformerEmbeddings(model_name="minishlab/potion-base-8M")

# Create Chroma vector store in memory
vectorstore = Chroma.from_documents(
    documents = documents,
    embedding = embedding_model,
    collection_name = "my_collection"
)


<ipython-input-20-1925534130>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="minishlab/potion-base-8M")


In [21]:
# Convert vectorStore into a retriever
retriever = vectorstore.as_retriever(search_kwargs={"k":2})

In [22]:
query = "What is Chroma used for?"
results = retriever.invoke(query)

In [23]:
# Print retrived content
for i, doc in enumerate(results):
  print(f"\n---Results {i+1} ----")
  print(f"Content:\n {doc.page_content}....")


---Results 1 ----
Content:
 Chroma is a vector database optimized for LLM-based search.....

---Results 2 ----
Content:
 Langchain helps developers build LLM application easily.....


#MMR

In [24]:
# Step 1: Your source documnets
docs = [
    Document(page_content="Langchain helps developers build LLM application easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embedding convert text into high-dimensional vectores."),
    Document(page_content="OpenAI provies powerful embedding models."),


]

In [25]:
from langchain_community.vectorstores import FAISS

In [26]:
# Initialize embedding model
embedding_model = SentenceTransformerEmbeddings(model_name="minishlab/potion-base-8M")


In [29]:
vectorstore = FAISS.from_documents(
    documents= docs,
    embedding = embedding_model
)

In [38]:
# Enable MMR in the retriever
# Enable MMR in the retriever
retriever = vectorstore.as_retriever(
    search_type = "mmr",
    search_kwargs={"k":3,"lambda_mult":1} # lambda value form 0 to 1
)

In [39]:
query = "What is langchain?"
results = retriever.invoke(query)

In [40]:
# Print retrived content
for i, doc in enumerate(results):
  print(f"\n---Results {i+1} ----")
  print(f"Content:\n {doc.page_content}....")


---Results 1 ----
Content:
 Langchain helps developers build LLM application easily.....

---Results 2 ----
Content:
 Chroma is a vector database optimized for LLM-based search.....

---Results 3 ----
Content:
 Embedding convert text into high-dimensional vectores.....
